In [1]:
%pdb

Automatic pdb calling has been turned ON


In [2]:
from __future__ import print_function
from __future__ import division
import torch
import sys
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
from PIL import Image
from torch import autograd
from torch.autograd import Variable
import scipy.misc
from sklearn.metrics import confusion_matrix
from torch.utils.data import Dataset, DataLoader
from torchvision.utils import make_grid
import pickle as pkl
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)
import tqdm
from tqdm import tqdm

PyTorch Version:  1.7.1
Torchvision Version:  0.8.2


In [3]:
data_file = "data/train_data/train_task_1_2.csv"
batch_size = 32
lr = 0.0001
num_epochs = 5
K = 100
total_q = 28000
total_s = 119000

In [4]:
df = pd.read_csv(data_file)

In [5]:
df.head()

,QuestionId,UserId,AnswerId,IsCorrect,CorrectAnswer,AnswerValue
0,16997,65967,12453206,0,4,2
1,16531,62121,15686710,1,1,1
2,15911,50013,13598796,0,3,1
3,1701,104909,10511925,0,4,3
4,22896,21748,941747,0,1,4


In [6]:
df.describe()

,QuestionId,UserId,AnswerId,IsCorrect,CorrectAnswer,AnswerValue
count,1.586785e+07,1.586785e+07,1.586785e+07,1.586785e+07,1.586785e+07,1.586785e+07
mean,1.380277e+04,5.952587e+04,9.916806e+06,6.429503e-01,2.502599e+00,2.488978e+00
std,7.967465e+03,3.428394e+04,5.725901e+06,4.791297e-01,1.104868e+00,1.094272e+00
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00
25%,6.890000e+03,2.980900e+04,4.957568e+06,0.000000e+00,2.000000e+00,2.000000e+00
50%,1.383000e+04,5.958500e+04,9.916784e+06,1.000000e+00,3.000000e+00,2.000000e+00
75%,2.069400e+04,8.912500e+04,1.487596e+07,1.000000e+00,3.000000e+00,3.000000e+00
max,2.761200e+04,1.189700e+05,1.983481e+07,1.000000e+00,4.000000e+00,4.000000e+00


In [7]:
df['QuestionId'].max()

27612

In [8]:
df['UserId'].max()

118970

In [9]:
class Question_Ans(Dataset):
    def __init__(self, filename, mode='train'):
        self.df = pd.read_csv(filename)
        self.questionid = self.df['QuestionId'].values
        self.userid = self.df['UserId'].values
        self.ans = self.df['IsCorrect'].values
        
        self.ans = 2*self.ans - 1
        
        self.length=len(self.ans)
        
        
        if(mode=='train'):
            start=int(0*self.length)
            end=int(0.8*self.length)
        else:
            start=int(0.8*self.length)
            end=int(1*self.length)
            
            
        self.questionid = self.questionid[start:end]
        self.userid = self.userid[start:end]
        self.ans = self.ans[start:end]
        
        self.length=len(self.ans)
        
    def __len__(self):
        return self.length
    
    def __getitem__(self, idx):
        
        qid = self.questionid[idx]
        uid = self.userid[idx]
        ans = self.ans[idx]
        return qid,uid,ans

In [10]:
train_dataset = Question_Ans(filename=data_file,mode='train')
val_dataset = Question_Ans(filename=data_file,mode='val')

In [11]:
train_dataloader = torch.utils.data.DataLoader(train_dataset,batch_size=batch_size, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_dataset,batch_size=batch_size, shuffle=False)

In [12]:
dataloader = {}
dataloader['train'] = train_dataloader
dataloader['val'] = val_dataloader

In [13]:
dtype=torch.FloatTensor

In [14]:
# Declare two matrices 
# Q = shape (total_q,K)
# U = shape (total_s,K)
Q = torch.nn.Embedding(total_q,K)
U = torch.nn.Embedding(total_s,K)

In [15]:
# embedding = torch.nn.Embedding(5,8)

In [16]:
# embedding(torch.LongTensor(np.array([3,4,3])))

In [17]:
def get_qvector(questions):
    
#     q_list = []
#     for i in range(questions.shape[0]):
#         q_list.append(Q[i])
    
#     return torch.cat(q_list,dim=0)
    ans = Q(torch.LongTensor(questions))
    
    return ans

In [18]:
def get_uvector(users):
    
#     u_list = []
#     for i in range(users.shape[0]):
#         u_list.append(U[i])
    
#     return torch.cat(u_list,dim=0)
#     print(users)
    ans = U(torch.LongTensor(users))
        
    return ans

In [19]:
test1 = torch.randn(32,100)
test2 = torch.randn(32,100)
test1 = torch.unsqueeze(test1,1)
test2 = torch.unsqueeze(test2,2)
print(test1.shape)
print(test2.shape)
res = torch.bmm(test1,test2)
print(res.shape)
res = torch.squeeze(res)
print(res.shape)

torch.Size([32, 1, 100])
torch.Size([32, 100, 1])
torch.Size([32, 1, 1])
torch.Size([32])


In [20]:
def get_score(qvectors,uvectors):
    
    q_unsq = torch.unsqueeze(qvectors, 1)
    u_unsq = torch.unsqueeze(uvectors, 2)
    score = torch.bmm(q_unsq,u_unsq)
    score = torch.squeeze(score)
    return score

In [21]:
params_to_update = list(Q.parameters()) + list(U.parameters())

In [22]:
criterion = nn.MSELoss()
optimizer = optim.SGD(params=params_to_update,lr=lr, momentum=0.9)

In [ ]:
best_loss = 1e9
for epoch in range(num_epochs):
    print('Epoch {}/{}'.format(epoch, num_epochs - 1))
    print('-' * 10)
    # Each epoch has a training and validation phase
    for phase in ['train', 'val']:
        
            
        running_loss = 0.0
        
        # Iterate over data.
        cnt = 0
#         print("*********** entered ",phase,"*********************")
        for questions,users,ans in tqdm(dataloader[phase]):
            
#             print("************ loaded data of one batch*******")
            
            cnt = cnt + 1
            ans = torch.tensor(ans)
            ans = ans.type(dtype)
            
            qvectors = get_qvector(questions)
            uvectors = get_uvector(users)
            
            
            # zero the parameter gradients
            optimizer.zero_grad()
            # forward
            
            scores = get_score(qvectors,uvectors)
            
            loss = criterion(scores,ans)
            
            if phase == 'train':
                loss.backward()
                optimizer.step()
        
            # statistics
            running_loss += loss.item()
            
            # compute correct
            # running_corrects += torch.sum(preds == labels.data)
            
            if (cnt % 1000 == 0 ) :
                print('Batch {} {} Loss: {:.4f} '.format(str(cnt),phase, running_loss/(cnt*batch_size)))
            

        epoch_loss = running_loss / len(dataloaders[phase].dataset)

        print(' {} Loss: {:.4f} '.format(phase, epoch_loss))
        
        # deep copy the model
        if phase == 'val' and epoch_loss < best_loss:
            best_loss = epoch_loss
            # best_model_wts = copy.deepcopy(model.state_dict())
            # do something about saving best matrices
    print()
time_elapsed = time.time() - since
print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
print('Best val Loss: {:4f}'.format(best_loss))

  0%|          | 0/396697 [00:00<?, ?it/s]

Epoch 0/4
----------


<ipython-input-23-94a917e17ba2>:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  ans = torch.tensor(ans)
  0%|          | 1004/396697 [00:52<5:13:28, 21.04it/s]

Batch 1000 train Loss: 3.1645 


  1%|          | 2002/396697 [01:40<5:08:32, 21.32it/s] 

...

  1%|          | 4004/396697 [03:18<5:19:31, 20.48it/s]

Batch 4000 train Loss: 3.1684 


  1%|▏         | 4999/396697 [04:24<7:44:40, 14.05it/s] 

Batch 5000 train Loss: 3.1559 


  1%|▏         | 5819/396697 [05:21<6:15:20, 17.36it/s] 